In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
# import matplotlib.pyplot as plt    # for plotting the images
# %matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [2]:
train_data = pd.DataFrame()
train_image = []
train_class = []
for i in range(17000):
    train_image.append('nude' + str(i))
    train_class.append('nude')
for i in range(12000):
    train_image.append('safe' + str(i))
    train_class.append('safe')
for i in range(7000):
    train_image.append('sexy' + str(i))
    train_class.append('sexy')
train_data['image'] = train_image
train_data['class'] = train_class
train_data.to_csv('train_new.csv',header=True, index=False)
train_data.head()

,image,class
0,nude0,nude
1,nude1,nude
2,nude2,nude
3,nude3,nude
4,nude4,nude


In [3]:
train_data['class'].drop_duplicates()

0        nude
17000    safe
29000    sexy
Name: class, dtype: object

In [4]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [5]:
train = pd.read_csv('train_new.csv')
train.shape

(36000, 2)

In [ ]:
from PIL import Image

In [6]:
# creating an empty list

train_image = []
# for loop to read and store frames

for i in tqdm(range(train.shape[0])):

    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('dataset/'+train['image'][i] + '.jpg', target_size=(80,80,3))

    # converting it to array
    img = image.img_to_array(img)

    # normalizing the pixel value
    img = img/255

    # appending the image to the train_image list
    train_image.append(img)

# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|████████████████████████████████████████████████████████████████████████████| 36000/36000 [06:45<00:00, 88.76it/s]


(36000, 80, 80, 3)

In [ ]:
import numpy as np
a = np.random.uniform(400, 800, size = (45000, 128, 128, 3))
print(a.size * a.itemsize)

In [7]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [8]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [9]:
y_train

,nude,safe,sexy
29179,0,0,1
33043,0,0,1
2586,1,0,0
20349,0,1,0
19146,0,1,0
...,...,...,...
23087,0,1,0
24171,0,1,0
2668,1,0,0
14252,1,0,0


In [10]:
import tensorflow as tf

In [11]:
# creating the base model of pre-trained VGG16 model
#base_model = VGG16(weights='imagenet', include_top=False,input_shape=(100,100,3))
base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(80,80,3))

In [12]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(28800, 2, 2, 512)

In [13]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(7200, 2, 2, 512)

In [14]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(28800, 2*2*512)
X_test = X_test.reshape(7200, 2*2*512)

In [15]:
# normalizing the pixel values
max = X_train.max()
print(max)
X_train = X_train/max
X_test = X_test/max

11.365967


In [16]:
X_train.shape

(28800, 2048)

In [17]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(2048,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [18]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight_80_36000.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [19]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [20]:
y_train = y_train.to_numpy(copy=False)
y_test = y_test.to_numpy(copy=False)

In [21]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [22]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
225/225 [==============================] - 14s 63ms/step - loss: 0.7755 - accuracy: 0.6670 - val_loss: 0.6728 - val_accuracy: 0.7206
Epoch 2/200
225/225 [==============================] - 13s 56ms/step - loss: 0.6539 - accuracy: 0.7308 - val_loss: 0.6078 - val_accuracy: 0.7415
Epoch 3/200
225/225 [==============================] - 12s 52ms/step - loss: 0.6165 - accuracy: 0.7501 - val_loss: 0.5978 - val_accuracy: 0.7462
Epoch 4/200
225/225 [==============================] - 12s 55ms/step - loss: 0.5865 - accuracy: 0.7611 - val_loss: 0.5887 - val_accuracy: 0.7518
Epoch 5/200
225/225 [==============================] - 12s 52ms/step - loss: 0.5696 - accuracy: 0.7683 - val_loss: 0.5770 - val_accuracy: 0.7592
Epoch 6/200
225/225 [==============================] - 12s 53ms/step - loss: 0.5545 - accuracy: 0.7763 - val_loss: 0.5768 - val_accuracy: 0.7644
Epoch 7/200
225/225 [==============================] - 12s 53ms/step - loss: 0.5360 - accuracy: 0.7823 - val_loss: 0.5798 - val_ac

225/225 [==============================] - 14s 60ms/step - loss: 0.1315 - accuracy: 0.9521 - val_loss: 1.1059 - val_accuracy: 0.7604
Epoch 58/200
225/225 [==============================] - 12s 55ms/step - loss: 0.1241 - accuracy: 0.9542 - val_loss: 1.1364 - val_accuracy: 0.7613
Epoch 59/200
225/225 [==============================] - 12s 51ms/step - loss: 0.1299 - accuracy: 0.9530 - val_loss: 1.0975 - val_accuracy: 0.7619
Epoch 60/200
225/225 [==============================] - 11s 49ms/step - loss: 0.1265 - accuracy: 0.9531 - val_loss: 1.1292 - val_accuracy: 0.7599
Epoch 61/200
225/225 [==============================] - 11s 50ms/step - loss: 0.1214 - accuracy: 0.9538 - val_loss: 1.1298 - val_accuracy: 0.7593
Epoch 62/200
225/225 [==============================] - 12s 54ms/step - loss: 0.1215 - accuracy: 0.9552 - val_loss: 1.1047 - val_accuracy: 0.7621
Epoch 63/200
225/225 [==============================] - 12s 52ms/step - loss: 0.1206 - accuracy: 0.9541 - val_loss: 1.2418 - val_accuracy

225/225 [==============================] - 12s 52ms/step - loss: 0.0767 - accuracy: 0.9719 - val_loss: 1.6000 - val_accuracy: 0.7667
Epoch 114/200
225/225 [==============================] - 12s 51ms/step - loss: 0.0729 - accuracy: 0.9744 - val_loss: 1.4237 - val_accuracy: 0.7615
Epoch 115/200
225/225 [==============================] - 12s 52ms/step - loss: 0.0765 - accuracy: 0.9736 - val_loss: 1.4208 - val_accuracy: 0.7638
Epoch 116/200
225/225 [==============================] - 11s 50ms/step - loss: 0.0749 - accuracy: 0.9739 - val_loss: 1.3408 - val_accuracy: 0.7640
Epoch 117/200
225/225 [==============================] - 12s 52ms/step - loss: 0.0821 - accuracy: 0.9706 - val_loss: 1.3937 - val_accuracy: 0.7594
Epoch 118/200
225/225 [==============================] - 12s 53ms/step - loss: 0.0796 - accuracy: 0.9723 - val_loss: 1.3168 - val_accuracy: 0.7608
Epoch 119/200
225/225 [==============================] - 12s 51ms/step - loss: 0.0703 - accuracy: 0.9750 - val_loss: 1.4296 - val_ac

Epoch 169/200
225/225 [==============================] - 11s 48ms/step - loss: 0.0594 - accuracy: 0.9797 - val_loss: 1.5195 - val_accuracy: 0.7639
Epoch 170/200
225/225 [==============================] - 11s 51ms/step - loss: 0.0582 - accuracy: 0.9799 - val_loss: 1.7792 - val_accuracy: 0.7667
Epoch 171/200
225/225 [==============================] - 11s 49ms/step - loss: 0.0608 - accuracy: 0.9783 - val_loss: 1.5359 - val_accuracy: 0.7633
Epoch 172/200
225/225 [==============================] - 11s 49ms/step - loss: 0.0564 - accuracy: 0.9810 - val_loss: 1.5330 - val_accuracy: 0.7683
Epoch 173/200
225/225 [==============================] - 11s 50ms/step - loss: 0.0549 - accuracy: 0.9816 - val_loss: 1.5357 - val_accuracy: 0.7636
Epoch 174/200
225/225 [==============================] - 11s 51ms/step - loss: 0.0588 - accuracy: 0.9791 - val_loss: 1.4333 - val_accuracy: 0.7658
Epoch 175/200
225/225 [==============================] - 11s 51ms/step - loss: 0.0535 - accuracy: 0.9811 - val_loss: 1

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3